# Introduction

TBD / Will cover:
- what is geospatial data?
- applications of geospatial data visualization
- pandas is a strong prereq for this course. in this tutorial, you'll get an idea if you need to review the pandas course before proceeding.
- set up the example with why we want to create a map of new york (idea being: [perhaps] you live in NYC and want to go hiking/camping in the country for a weekend. instead of using TripAdvisor to guide your trip, you're excited about all of the data that the state of NY has released and want to do your own, tailored analysis based on your specific preferences)

# Reading and plotting (shapefile) data

The first step is to read in the data!  To do this, we'll use the [Geopandas](http://geopandas.org/) library.

In [ ]:
import geopandas as gpd

Geopandas is an extension of the [Pandas](https://pandas.pydata.org/) library with added functionality for geospatial data.

With Geopandas, we can read data from a variety of common geospatial file formats, such as [shapefile](https://en.wikipedia.org/wiki/Shapefile), [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language), and [GPKG](https://en.wikipedia.org/wiki/GeoPackage).  You can also work with [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) files.  

For now, we'll load a shapefile containing information about lands under the care of the [Department of Environmental Conservation](https://www.dec.ny.gov/index.html) in the state of New York.

In [ ]:
# Read in the data
full_data = gpd.read_file("../input/geospatial-course-data/dec_lands/DEC_lands.shp")
full_data.head()

The command above reads the data into a (GeoPandas) **GeoDataFrame** object that has all of the capabilities of a (Pandas) DataFrame object: so, everything you learned in the [Pandas micro-course](https://www.kaggle.com/learn/pandas) can be used to work with the data. 

In [ ]:
type(full_data)

For instance, if we don't plan to use all of the columns, we can select a subset of them.  
> To review other methods for selecting data, check out [this tutorial](https://www.kaggle.com/residentmario/indexing-selecting-assigning/) from the Pandas micro-course.

In [ ]:
data = full_data.loc[:, ["CLASS", "COUNTY", "geometry"]].copy()

We use the `value_counts()` method to see a list of different land types, along with how many times they appear in the dataset. 
> To review this (and related methods), check out [this tutorial](https://www.kaggle.com/residentmario/summary-functions-and-maps) from the Pandas micro-course.

In [ ]:
# How many lands of each type are there?
data.CLASS.value_counts()

You can also use `loc` (and `iloc`) and `isin` to select subsets of the data.  
> To review this, check out [this tutorial](https://www.kaggle.com/residentmario/indexing-selecting-assigning/) from the Pandas micro-course.

In [ ]:
# Select lands that fall under the "WILD FOREST" or "WILDERNESS" category
wild_lands = data.loc[data.CLASS.isin(['WILD FOREST', 'WILDERNESS'])].copy()
wild_lands.head()

GeoDataFrames also have some added methods and attributes (that don't apply to DataFrames).  For instance, we can quickly visualize the data with the `plot()` method.  This method takes as (optional) input several parameters that can be used to customize the appearance of your plots.

In [ ]:
# Other available options line up with matplotlib options
wild_lands.plot(color='darkgreen')

# The "geometry" column

Every GeoDataFrame contains a special "geometry" column.  It contains all of the geometric objects that are displayed when we call the `plot()` method.

In [ ]:
wild_lands.geometry.head()

While this column can contain a variety of different datatypes, each entry will typically be a `Point`, `LineString`, or `Polygon`.  The "geometry" column in the dataset that we've just loaded contains 2983 different `Polygon` objects, each corresponding to a different shape in the plot above.

In general, for an arbitrary dataset, the type in the "geometry" column will depend on what we are trying to show: for instance, we might use a `Point` for the epicenter of an earthquake, a `Line` for a street, and a `Polygon` to show country boundaries.

![](https://i.imgur.com/N1llefr.png)

All three types of geometric objects have built-in attributes that you can use to quickly analyze the dataset.  For instance, you can get the area of a polygon from the `area` attribute.

In [ ]:
# Calculate the area (in square meters) of each polygon in the GeoDataFrame 
wild_lands.loc[:, "AREA"] = wild_lands.geometry.area 
wild_lands.head()

With this information, we can check how many square kilometers of forest and wilderness can be found in the state of New York.  
> If you need to review how to use `groupby()`, check out [this tutorial](https://www.kaggle.com/residentmario/grouping-and-sorting) from the Pandas micro-course.

In [ ]:
# Divide by 10**6 to convert square meters to square kilometers
wild_lands.groupby('CLASS').AREA.sum() / 10**6

New York has approximately 6193 square kilometers of forest, and 5494 square kilometers of wilderness.

# Coordinate reference system (CRS)

All of the maps you will create in this course portray the surface of the earth in two dimensions.  But, as you know, the world is actually a three-dimensional globe, and so we have to use a special method, called a **map projection**, to render it as a flat surface.  No map projection can ever be 100% accurate.  But, depending on the regional extent of your map and the type of analysis you plan to do, some are better than others.

We use a **coordinate reference system (CRS)** to show how the points on the map correspond to real locations on Earth.  And, we identify different coordinate reference systems according to their [European Petroleum Survey Group (EPSG)](http://www.epsg.org/) codes.  For instance, EPSG code 4326 corresponds to coordinates in latitude and longitude.

We can get the CRS of any GeoDataFrame from the `crs` attribute.

In [ ]:
# Check coordinate reference system (CRS)
wild_lands.crs

From here, we can see that the GeoDataFrame has a CRS of [EPSG 26918](https://epsg.io/26918), which is a good choice for mapping geospatial data based in New York.  While we won't go into how to select an appropriate CRS for your data in this micro-course, you can read more about this at [this link](https://www.axismaps.com/guide/general/map-projections/).  

When plotting data from multiple GeoDataFrames, it is important that they all have the same CRS.  In the code cell below, we create three more GeoDataFrames, containing campsite locations (`Point`), foot trails (`LineString`), and county boundaries (`Polygon`).

In [ ]:
# Contains locations of campsites in New York state (Point)
POI_data = gpd.read_file("../input/geospatial-course-data/dec_pointsinterest/Decptsofinterest.shp")
campsites = POI_data.loc[POI_data.ASSET=='PRIMITIVE CAMPSITE'].copy()

# Contains foot trails in New York state (LineString)
roads_trails = gpd.read_file("../input/geospatial-course-data/dec_roadstrails/Decroadstrails.shp")
trails = roads_trails.loc[roads_trails.ASSET=='FOOT TRAIL'].copy()

# Contains county boundaries in New York state (Polygon)
counties = gpd.read_file("../input/geospatial-course-data/cugir-007865/cugir-007865/cty036.shp")

And, below, we check the CRS of each one.

In [ ]:
print(counties.crs)
print(campsites.crs)
print(trails.crs)

The `counties` GeoDataFrame has a CRS that differs from the others!  To fix this, we use the `to_crs()` method to transform the CRS to EPSG 26918.

In [ ]:
ax = counties.to_crs(epsg=26918).plot(figsize=(10,10), color='none', edgecolor='gainsboro', zorder=3)
wild_lands.plot(color='lightgreen', ax=ax)
campsites.plot(color='maroon', markersize=2, ax=ax)
trails.plot(color='black', markersize=1, ax=ax)

And now we have a map of New York that we can use to guide our next trip!

# Your turn

Use what you've learned to investigate ...